## Extact planet data

In [1]:
# Importar las bibliotecas necesarias
import geemap
import ee
import geopandas as gpd
import json
import pandas as pd

In [2]:
# Autenticación y inicialización de GEE
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [2]:
# Configurar el mapa
Map = geemap.Map(center=(12.20, -16.36), zoom=12)
Map

Map(center=[12.2, -16.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
Map.user_roi.getInfo()

In [4]:
# Asegurarnos de la creación de una geometría
geometry = ee.Geometry.Polygon([[[-16.453217, 12.240196],
   [-16.453217, 12.251142],
   [-16.427029, 12.251142],
   [-16.427029, 12.240196],
   [-16.453217, 12.240196]]])

Indices a añadir: 

EVI (bryan)
LSWI
NDSI
RNDVI
NDRE
MNDWI
RVI
NDMI

NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI

In [18]:
# Cargar el shapefile
shapefile_path = '../DataIn/Parcelas_poligono.shp'
gdf = gpd.read_file(shapefile_path)

# Convertir el GeoDataFrame a GeoJSON y luego a FeatureCollection
def geojson_to_ee(gdf):
    geojson = json.loads(gdf.to_json())
    features = [ee.Feature(ee.Geometry(feature['geometry']), feature['properties']) for feature in geojson['features']]
    return ee.FeatureCollection(features)

fc = geojson_to_ee(gdf)


Map.addLayer(fc)

# Definir la colección de imágenes y el área de interés
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Imagenes_planet_folder/Collection_planet") \
              .filterDate('2022-07-01', '2024-01-30') \
              .filterBounds(geometry)


# Función para recortar imágenes
def clip_image(image):
    return image.clip(geometry)

# Aplicar la función de recorte a cada imagen en la colección
collection = collection.map(clip_image)

# Función para añadir índices espectrales
def addIndices(img):
    # Cálculo de índices existentes
    NDVI = img.normalizedDifference(['nir', 'red']).rename('NDVI')
    SR = img.select('nir').divide(img.select('red')).rename('SR')
    GCVI = img.expression('(NIR / GREEN) - 1', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GCVI')
    NDWI = img.expression('(GREEN - NIR) / (GREEN + NIR)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('NDWI')
    VARI = img.expression('(GREEN - RED) / (GREEN + RED - BLUE)', {'BLUE': img.select('blue'), 'RED': img.select('red'), 'GREEN': img.select('green')}).rename('VARI')
    GRVI = img.expression('NIR / GREEN', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GRVI')
    GNDVI = img.normalizedDifference(['nir', 'green']).rename('GNDVI')
    SAVI = img.expression('(NIR - RED) / ((NIR + RED + 0.5) * 1.5)', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('SAVI')
    # Índices adicionales que utilizan el borde rojo
    NDRE = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDRE')
    NDVIre = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDVIre')
    REI = img.expression('(RED_EDGE - RED) / (RED_EDGE + RED)', {'RED_EDGE': img.select('rededge'), 'RED': img.select('red')}).rename('REI')
    
    # Añadiendo todos los índices calculados a la imagen
    return img.addBands([
        NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI, 
        NDRE, NDVIre, REI
    ])

# Aplicar la función a la colección de imágenes
ps_1 = collection.map(addIndices)

# Crear un composite de las imágenes
composite = ps_1.mean().float()

# Añadir la capa al mapa
Map.addLayer(composite, {'bands': ['nir', 'red', 'green'], 'min': 201, 'max': 2464}, 'RGB')
Map.addLayer(geometry)


In [19]:
# Convertir el GeoDataFrame a GeoJSON y luego a FeatureCollection
def geojson_to_ee(gdf):
    geojson = json.loads(gdf.to_json())
    features = [ee.Feature(ee.Geometry(feature['geometry']), feature['properties']) for feature in geojson['features']]
    return ee.FeatureCollection(features)

fc = geojson_to_ee(gdf)


In [20]:

# Extraer datos para cada polígono y cada imagen
def extract_data(image):
    def zonal_stats(feature):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=3,  #
            maxPixels=1e9
        )
        return feature.set(stats).set({'image_date': image.date().format('YYYY-MM-dd')})
    
    return fc.map(zonal_stats)

# Iterar sobre la colección y extraer datos
results = []
image_list = collection.toList(collection.size())

for i in range(collection.size().getInfo()):
    image = ee.Image(image_list.get(i))
    image = addIndices(image)
    data = extract_data(image).getInfo()
    
    for feature in data['features']:
        properties = feature['properties']
        properties['geometry'] = feature['geometry']
        results.append(properties)

In [21]:
# Convertir los resultados a un DataFrame de pandas
df = pd.DataFrame(results)

# Exportar a un archivo CSV
df.to_csv('../Results/output_indices_planet.csv', index=False)

print('Exportación completada.')

Exportación completada.
